In [7]:
import pandas as pd
import os
from glob import glob
import numpy as np
import json
from itertools import izip

In [8]:
def get_env_var(handle):
    ''' Get an environment variable given the handle for the bash variable'''
    tmp = os.getenv(handle)
    if not tmp:
        raise LookupError("Environment variable: {} not set.".format(handle))
    return tmp.strip("'")

def format_md_img(link_name, rel_plot_loc):
    img_formatter="[[{}/{}]]".format(link_name, rel_plot_loc)
    return img_formatter

def print_table(list_of_headers):
    fmter =  "| {} {}".format("{}", " {} ")
    for header in list_of_headers:
        fmter = fmter.format("{} {}".format(header, "| {}"), "{}")
    fmter = fmter.format(" ", " ")
    return fmter

In [9]:
class  Experiment(dict):
    def __init__(self, name):
        self.name = name
        self.folds = []
        self.converged_epochs = []
        self.trained_paths = []
      
    def __repr__(self):
        return str(vars(self))
    
    def __str__(self):
        return json.dumps(vars(self), indent=2)
        
    def set_convereged_epoch(self, epoch, train_path, fold=None):        
        self.folds.append(fold)
        self.converged_epochs.append(epoch)
        self.trained_paths.append(train_path)
        

In [10]:
home=get_env_var("HOME")
base="{}/labgits/lab-notebook-caceres.wiki/images/20180815_Paper_Retrains_classification_lr01".format(home)

if not os.path.exists(base):
    os.mkdir(base)

`rsync -Rnv $HOME_SAVE_BASE/output/20180815_Paper_Retrains/trained_nets/./*/*_LR01/*/*/*.png $HOME/labgits/lab-notebook-caceres.wiki/images/20180815_Paper_Retrains_classification_lr01/`

In [11]:
home_save_dir = get_env_var("HOME_SAVE_BASE")
srv_save_dir = get_env_var("DATA_SAVE_BASE")

fold_tmplt = "fold_[0-9]*/pnr_*/*.png"

# github formatting info:
github_wiki_link="https://github.com/keiserlab/lab-notebook-caceres/wiki/images"
expt_name = "20180815_Paper_Retrains_classification_lr01"
github_wiki_expt = "{}/{}".format(github_wiki_link, expt_name)
train_dir = "{}//lr_nesterov_binary_classifier_1024_2048_3072/".format(base)

In [12]:
expt_names = ["CLASSIFIER_SMA_RATIOS_LR01" , "CLASSIFIER_NEG_RM_SMA_RATIOS_LR01"]
sneg_pnrs=[1.0, 1.2222, 0.8182, 1.5, 19.0, 9.0, 4.0, 2.3333, 0.6666, 0.4286, 0.25, 0.1111, 0.0753]
sneg_pnrs = sorted(sneg_pnrs)

for k in expt_names:
    print("# {}".format(k))
    parent_dir = os.path.join(train_dir, k)
    if not os.path.exists(parent_dir):
        os.mkdir(parent_dir)
    png_files = glob(os.path.join(parent_dir, fold_tmplt))

    experiments = []
    converged_epochs = ("{}/output/20180815_Paper_Retrains/predictions/{}/experiments.json".format(home_save_dir, k))
    with open(converged_epochs, "r") as fp:
        expts = json.load(fp)

    expt_list = []
    for e in expts:
        tmp = Experiment(e["name"])
        for epoch, path, fold in izip(e["converged_epochs"], e["trained_paths"], e["folds"]):
            tmp.set_convereged_epoch(epoch, path, fold)
        experiments.append(tmp)
        del(tmp)    

    expt_dict = {float(i.name.split("_")[-1]): dict(zip([int(j.split("_")[1]) for j in i.folds], i.converged_epochs))
                 for i in experiments}
    img_order = ['train_loss','test_loss', 'test_no_gt_loss', 'test_sneg_loss']
    for ratio in sneg_pnrs:
        png_subset = [f for f in png_files if str(ratio) in f]
        headers = ["PNR_{}".format(ratio)]
        headers.extend(img_order)
        print(print_table(headers))
        print(print_table([":---"]*len(headers)))
        for f in np.arange(0, 5, 1):
            name_fmter = "fold_{} best epoch: {}".format(str(f), expt_dict[ratio][f])
            fold_fmter = "fold_{}".format(str(f))
            fold_pngs = [i for i in png_subset if fold_fmter in i]
            fold_pngs = sorted(fold_pngs, key=lambda x: img_order.index(x.split("/")[-1].split(".")[0]))
            md_print_fmt = [name_fmter] + [format_md_img(github_wiki_expt, i.split("{}/".format(expt_name))[-1]) for i in fold_pngs]
            print(print_table(md_print_fmt))
        print("\n\n")

# CLASSIFIER_SMA_RATIOS_LR01


OSError: [Errno 2] No such file or directory: '/srv/home/ecaceres/labgits/lab-notebook-caceres.wiki/images/20180815_Paper_Retrains_classification_lr01//lr_nesterov_binary_classifier_1024_2048_3072/CLASSIFIER_SMA_RATIOS_LR01'

In [32]:
expt_names = ["CLASSIFIER_STD_LR01", "CLASSIFIER_SEA_SMA_LR01", "CLASSIFIER_scrambled_idx_LR01", "CLASSIFIER_scrambled_idx_No_SMA_LR01", ]
sneg_pnrs=[1.0]
sneg_pnrs = sorted(sneg_pnrs)
train_dir = "{}/lr_nesterov_binary_classifier_1024_2048_3072/".format(base)

for k in expt_names:
    print("# {}".format(k))
    parent_dir = os.path.join(train_dir, k)
    if not os.path.exists(parent_dir):
        os.mkdir(parent_dir)
    png_files = glob(os.path.join(parent_dir, fold_tmplt))

    experiments = []
    converged_epochs = ("{}/output/20180815_Paper_Retrains/predictions/{}/experiments.json".format(home_save_dir, k))
    with open(converged_epochs, "r") as fp:
        expts = json.load(fp)

    expt_list = []
    for e in expts:
        tmp = Experiment(e["name"])
        for epoch, path, fold in izip(e["converged_epochs"], e["trained_paths"], e["folds"]):
            tmp.set_convereged_epoch(epoch, path, fold)
        experiments.append(tmp)
        del(tmp)    

    expt_dict = {float(i.name.split("_")[-1]): dict(zip([int(j.split("_")[1]) for j in i.folds], i.converged_epochs))
                 for i in experiments}
    img_order = ['train_loss','test_loss', 'test_no_gt_loss', 'test_sneg_loss']
    for ratio in sneg_pnrs:
        png_subset = [f for f in png_files if str(ratio) in f]
        headers = ["PNR_{}".format(ratio)]
        headers.extend(img_order)
        print(print_table(headers))
        print(print_table([":---"]*len(headers)))
        
        for f in np.arange(0, 5, 1):
            name_fmter = "fold_{} best epoch: {}".format(str(f), expt_dict[ratio][f])
            fold_fmter = "fold_{}".format(str(f))
            fold_pngs = [i for i in png_subset if fold_fmter in i]
            fold_pngs = sorted(fold_pngs, key=lambda x: img_order.index(x.split("/")[-1].split(".")[0]))
            md_print_fmt = [name_fmter] + [format_md_img(github_wiki_expt, i.split("{}/".format(expt_name))[-1]) for i in fold_pngs]
            print(print_table(md_print_fmt))
        print("\n\n")

# CLASSIFIER_SEA_SMA
| PNR_1.0 | train_loss | test_loss | test_no_gt_loss | test_sneg_loss |      
| :--- | :--- | :--- | :--- | :--- |      
| fold_0 best epoch: 0 | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains_classification/lr_nesterov_binary_classifier_1024_2048_3072/CLASSIFIER_SEA_SMA/fold_0/pnr_1.0/train_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains_classification/lr_nesterov_binary_classifier_1024_2048_3072/CLASSIFIER_SEA_SMA/fold_0/pnr_1.0/test_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains_classification/lr_nesterov_binary_classifier_1024_2048_3072/CLASSIFIER_SEA_SMA/fold_0/pnr_1.0/test_no_gt_loss.png]] | [[https://github.com/keiserlab/lab-notebook-caceres/wiki/images/20180815_Paper_Retrains_classification/lr_nesterov_binary_classifier_1024_2048_3072/CLASSIFIER_SEA_SMA/fold_0/pnr_1.0/test_sneg_loss.png]] |      
| fold_1 best e